In [1]:
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master('local[*]') \
    .appName('test') \
    .getOrCreate()

24/03/04 15:53:47 WARN Utils: Your hostname, pedro resolves to a loopback address: 127.0.1.1; using 192.168.15.20 instead (on interface wlp5s0)
24/03/04 15:53:47 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/03/04 15:53:48 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/03/04 15:53:49 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [25]:
df_green = spark.read.parquet('data/pq/green/*/*')
df_yellow = spark.read.parquet('data/pq/yellow/*/*')

In [35]:
# df_green.registerTempTable('green') # deprecate warning
df_green.createOrReplaceTempView('green')
df_yellow.createOrReplaceTempView('yellow')


In [27]:
df_green_revenue = spark.sql("""
SELECT 
    -- Reveneue grouping 
    date_trunc('hour', lpep_pickup_datetime) AS hour, 
    PULocationID AS zone,
    
    SUM(total_amount) AS amount,
    COUNT(1) AS number_records

FROM
    green
WHERE
    lpep_pickup_datetime >= '2020-01-01 00:00:00'
GROUP BY
    1, 2
ORDER BY
    1, 2

""")

In [28]:
df_yellow_revenue = spark.sql("""
SELECT 
    -- Reveneue grouping 
    date_trunc('hour', tpep_pickup_datetime) AS hour, 
    PULocationID AS zone,
    
    SUM(total_amount) AS amount,
    COUNT(1) AS number_records

FROM
    yellow
WHERE
    tpep_pickup_datetime >= '2020-01-01 00:00:00'
GROUP BY
    1, 2
ORDER BY
    1, 2

""")

In [ ]:
df_green_revenue.show()


In [ ]:
df_yellow_revenue.show()

In [33]:
df_green_revenue \
    .repartition(20) \
    .write.parquet('data/report/revenue/green', mode='overwrite')


24/03/04 16:32:57 WARN MemoryManager: Total allocation exceeds 95,00% (1.020.054.720 bytes) of heap memory
Scaling row group sizes to 95,00% for 8 writers
24/03/04 16:32:58 WARN MemoryManager: Total allocation exceeds 95,00% (1.020.054.720 bytes) of heap memory
Scaling row group sizes to 95,00% for 8 writers


In [34]:
df_yellow_revenue \
    .repartition(20) \
    .write.parquet('data/report/revenue/yellow', mode='overwrite')

24/03/04 16:34:05 WARN MemoryManager: Total allocation exceeds 95,00% (1.020.054.720 bytes) of heap memory
Scaling row group sizes to 95,00% for 8 writers
24/03/04 16:34:05 WARN MemoryManager: Total allocation exceeds 95,00% (1.020.054.720 bytes) of heap memory
Scaling row group sizes to 95,00% for 8 writers


In [38]:
df_green_revenue_tmp = df_green_revenue \
    .withColumnRenamed('amount','green_amount') \
    .withColumnRenamed('number_records','green_number_records')

df_yellow_revenue_tmp = df_yellow_revenue \
    .withColumnRenamed('amount','yellow_amount') \
    .withColumnRenamed('number_records','yellow_number_records')

In [39]:
# joining green with yellow
df_join = df_green_revenue_tmp.join(df_yellow_revenue_tmp, on=['hour','zone'], how='outer')

In [41]:
df_join.columns

['hour',
 'zone',
 'green_amount',
 'green_number_records',
 'yellow_amount',
 'yellow_number_records']

In [40]:
df_join.show()

+-------------------+----+------------------+--------------------+------------------+---------------------+
|               hour|zone|      green_amount|green_number_records|     yellow_amount|yellow_number_records|
+-------------------+----+------------------+--------------------+------------------+---------------------+
|2020-01-01 00:00:00|  29|              61.3|                   1|              null|                 null|
|2020-01-01 00:00:00|  41|1363.9599999999984|                  84|           1256.53|                   80|
|2020-01-01 00:00:00|  55|129.29000000000002|                   4|              null|                 null|
|2020-01-01 00:00:00|  65|199.48999999999998|                  10| 409.3500000000001|                   19|
|2020-01-01 00:00:00|  72|             67.53|                   2|              null|                 null|
|2020-01-01 00:00:00|  83|              94.1|                   7|               9.8|                    1|
|2020-01-01 00:00:00| 122|10

In [42]:
df_zones = spark.read.parquet('zones/')

In [43]:
df_result = df_join.join(df_zones, df_join.zone == df_zones.LocationID)

In [45]:
df_result.drop('LocationID', 'zone') \
    .write.parquet('tmp/revenue-zones')

24/03/04 20:07:05 WARN MemoryManager: Total allocation exceeds 95,00% (1.020.054.720 bytes) of heap memory
Scaling row group sizes to 95,00% for 8 writers
